# 单个动作截取

In [39]:
# 存储单个动作波形，命名为user02_fluorescent_xxx【xxx为class_num】
def save_single_action(file_name,start,end,class_num):
    # 得到单个动作
    df = pd.read_csv(f'../../event_csv/split_data/class{class_num}/{file_name}.csv').query(f'timestamp >= {start} & timestamp < {end}')
    pd.DataFrame([['timestamp','x','y']]).to_csv(
    f'../../event_csv/split_data/artificial/{file_name}_{class_num}.csv',
    index=False,header=False,mode='w')
    df.to_csv(
    f'../../event_csv/split_data/artificial/{file_name}_{class_num}.csv',
    index=False,header=False,mode='a'
    )

# 相同动作合成

In [41]:
# 合成名字为{file_name}.csv文件指定时间戳[start,end]的动作repeat_times次
# 返回单个动作事件个数，方便往重复动作中插入其他动作
def get_same(file_name,class_num,repeat_times):
    # 得到单个动作
    df = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}_{class_num}.csv')
    # 合成repeat_times次
    df1 = df 
    for i in range(repeat_times-1):
        df1 = pd.concat([df1, df],ignore_index=True)
    pd.DataFrame([['timestamp','x','y']]).to_csv(
    f'../../event_csv/split_data/artificial/{file_name}_{class_num}_same.csv',
    index=False,header=False,mode='w'
    )
    df1.to_csv(
    f'../../event_csv/split_data/artificial/{file_name}_{class_num}_same.csv',
    index=False,header=False,mode='a'
    )

# 相同动作拼接
> 统一将class2视为动作A，class7视为动作B

In [1]:
# 将动作B放在动作A之前/后【在此之前A已合成5次，B合成了3次】，B_front_flag为True代表B在前面
def get_front_or_tail(file_name,class_num_A,class_num_B):
    # 动作A
    df_A = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}_{class_num_A}_same.csv')
    # 动作B
    df_B = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}_{class_num_B}_same.csv')

    pd.DataFrame([['timestamp','x','y']]).to_csv(f'../../event_csv/split_data/artificial/{file_name}_front.csv',
                                     index=False,header=False,mode='w')
    # B接在A前面
    pd.concat([df_B,df_A],ignore_index=True).to_csv(f'../../event_csv/split_data/artificial/{file_name}_front.csv',
                                     index=False,header=False,mode='a')

    pd.DataFrame([['timestamp','x','y']]).to_csv(f'../../event_csv/split_data/artificial/{file_name}_tail.csv',
                                     index=False,header=False,mode='w')
    # B接在A后面
    pd.concat([df_A,df_B],ignore_index=True).to_csv(f'../../event_csv/split_data/artificial/{file_name}_tail.csv',
                                         index=False,header=False,mode='a')

In [49]:
# 将动作B插入到第behind_A个A动作之后
def get_mid(file_name,class_num_A,class_num_B,behind_A):
    # 单个A动作长度
    A_singal_length = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}_{class_num_A}.csv').shape[0]
    # 动作A
    df_A = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}_{class_num_A}_same.csv')
    # 动作B
    df_B = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}_{class_num_B}_same.csv')
    # 动作A分界点
    gap_point = A_singal_length * behind_A
    pd.DataFrame([['timestamp','x','y']]).to_csv(f'../../event_csv/split_data/artificial/{file_name}_mid.csv',
                                         index=False,header=False,mode='w')
    
    pd.concat([df_A[:gap_point],df_B,df_A[gap_point:]],ignore_index=True).to_csv(f'../../event_csv/split_data/artificial/{file_name}_mid.csv',
                                         index=False,header=False,mode='a')

# 人工合成数据方法整合

In [50]:
def get_syn_data(file_name='user02_fluorescent',start_A=-1,end_A=-1,class_num_A=2,repeat_times_A = 5,
                start_B=-1,end_B=-1,class_num_B=7,repeat_times_B = 3):
    # 保存单一动作
    save_single_action(file_name,start_A,end_A,class_num_A)
    save_single_action(file_name,start_B,end_B,class_num_B)
    
    # 合成相同动作A
    get_same(file_name,class_num_A,repeat_times_A)
    # 合成相同的动作B 
    get_same(file_name,class_num_B,repeat_times_B)
    
    # 多个B拼接在A前/后面
    get_front_or_tail(file_name,class_num_A,class_num_B)
    
    # 将多个B拼接在第三个A之后
    get_mid(file_name,class_num_A,class_num_B,behind_A=3)
    